# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *

import os
from Utils import cacheOnDisk
from queue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

from Levenshtein import distance
from scipy.spatial.distance import pdist, squareform

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
PNP530Cardio126=PNP530+Cardio126


done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!


/usr/wisdom/python/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


end of function!!!


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'28112018'

# plan:

1. load the unique aa table and generate something similar but very small. (enable selection of productive/non productive or both).
2. generate distance matrix with metric=hamming distance
3. new matrix: 1 for each cell with value<2 in old matrix. change col names to c1,c2..... 
4. filter out columns with sum=1
5. calculate euclidean distance between remaining columns, in a pair with dist=1 or -1 or 0, delete the second cluster (check this!!!)
6. end up with cluster matrix: seqeunces by clusters (c1,c2...) and values represent 'appear in cluster'
7. this matrix can be then merge with each sample sequence list to yield a df showing which clusters are present in the sample.
8. generate 1-row df for each sample, with 1 for any clusters that appear in the sample.
9. concat all dfs, to get a matrix of samples X clusters. 

10 .this procedures should be modular so I can change the following parameters: initial sample folder (defines the dataset or defined by the dataset name), hamming distance threshold <2 or <3, info to collect - presence/absence of cluster or also frequency (for frequency, need to add a stage after 7 in which a new df is generated, and for each '1' in the former df, we extract the frequency of the sequence in the sample. that in #8, we summarize the frequencies and not just marking 1. 

# get all productive sequences:

In [3]:
f1='%s/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/AllUniqueWithCounts' %MyPath
AllUniqueWithCounts=pd.read_pickle(f1)

In [4]:
print AllUniqueWithCounts.shape

(7864153, 5)


In [5]:
AllUniqueWithCounts.head()

,frequencyCount (%),prod_stat,Sample,nShared,isPublic
***ADGPYEQYV,0.009748,0,BD463,1,0
**AGGEQYF,0.003249,0,BD856,1,0
**EDKSGPQYF,0.003275,0,BD944,1,0
**EQFF,0.003405,0,BD845,1,0
**ERWVPDTQYF,0.003868,0,BD565,1,0


In [6]:
UniqueSeqProd=AllUniqueWithCounts[AllUniqueWithCounts['prod_stat']==1]
print UniqueSeqProd.shape

(7700449, 5)


In [9]:
UniqueSeqNonProd=AllUniqueWithCounts[AllUniqueWithCounts['prod_stat']==0]
print UniqueSeqNonProd.shape
print UniqueSeqNonProd['Sample'].nunique()

(163704, 5)
656


# calculate hamming distnace on all non-productive sequences:

In [10]:
def gen_hamming_square_distMat(seqs):
    if type(seqs)==list:
        print 'seqs type is a list'
    elif type(seqs)==pd.core.frame.DataFrame:
        try:
            seqs=seqs.index.tolist()
            print 'seqs type is a dataframe'
        except: 
            print 'couldnt identify seqs type'
            return




    # prepare 2 dimensional array M x N (M entries (3) with N dimensions (1)) 
    transformed_seqs = np.array(seqs).reshape(-1,1)

    # calculate condensed distance matrix by wrapping the Levenshtein distance function
    distance_matrix = pdist(transformed_seqs,lambda x,y: distance(x[0],y[0]))

    # get square matrix
    # print distance_matrix
    distance_df=pd.DataFrame(squareform(distance_matrix),index=seqs,columns=seqs)
    print 'distMat header:'
    print distance_df.iloc[:4,:4]
    
    return distance_df

In [11]:
hamming_distMat_allNonProd=gen_hamming_square_distMat(UniqueSeqNonProd)

seqs type is a dataframe


KeyboardInterrupt: 

In [ ]:
f2='%s/TCR_real_data/PNP530Cardio126Combined/sharingAnalysis/hamming_distMat_allNonProd' %MyPath
hamming_distMat_allNonProd.to_pickle(f2)